In [1]:
# libraries

from datetime import datetime
import os
import glob
import requests 
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Web scrapping

In [2]:
# download data 
# =============

link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSc_2y5N0I67wDU38DjDh35IZSIS30rQf7_NYZhtYYGU1jJYT6_kDx4YpF-qw0LSlGsBYP8pqM_a1Pd/pubhtml#'
req = requests.get(link)
soup = BeautifulSoup(req.content, "html.parser")

In [3]:
tbody = soup.find_all('tbody')[0]
body = tbody.find_all('tr')

# print(tbody)
# print(body)

In [4]:
head_row = [i.text for i in body[0].find_all('td')]
head_row

['Patient Number',
 'State Patient Number',
 'Date Announced',
 'Age Bracket',
 'Gender',
 'Detected City',
 'Detected District',
 'Detected State',
 'State code',
 'Current Status',
 'Notes',
 'Contracted from which Patient (Suspected)',
 'Nationality',
 'Type of transmission',
 'Status Change Date',
 'Source_1',
 'Source_2',
 'Source_3',
 'Backup Notes',
 '',
 '',
 '',
 '',
 '',
 '',
 '']

In [5]:
contents = []

for i in range(len(body)):
    contents.append([i.text for i in body[i].find_all('td')])

# Saving to Dataframe

In [6]:
p_df = pd.DataFrame(contents[2:len(contents)], columns=head_row)
p_df.head()

Patient Number State Patient Number Date Announced Age Bracket Gender  \
0              1             KL-TS-P1     30/01/2020          20      F   
1              2             KL-AL-P1     02/02/2020                      
2              3             KL-KS-P1     03/02/2020                      
3              4                DL-P1     02/03/2020          45      M   
4              5                TS-P1     02/03/2020          24      M   

              Detected City Detected District Detected State State code  \
0                  Thrissur          Thrissur         Kerala         KL   
1                 Alappuzha         Alappuzha         Kerala         KL   
2                 Kasaragod         Kasaragod         Kerala         KL   
3  East Delhi (Mayur Vihar)        East Delhi          Delhi         DL   
4                 Hyderabad         Hyderabad      Telangana         TG   

  Current Status  ...                                           Source_2  \
0      Recovered  ...  https://weather.com/en-IN/india/news/news/2020...   
1      Recovered  ...  https://weather.com/en-IN/india/news/news/2020...   
2      Recovered  ...  https://twitter.com/ANI/status/122422148580539...   
3      Recovered  ...  https://economictimes.indiatimes.com/news/poli...   
4      Recovered  ...  https://www.indiatoday.in/india/story/coronavi...   

                                            Source_3  \
0                                 Student from Wuhan   
1                                                      
2  https://weather.com/en-IN/india/news/news/2020...   
3                                                      
4  https://www.thehindu.com/news/national/coronav...   

                                 Backup Notes                    
0                                                          None  
1                          Student from Wuhan                    
2                          Student from Wuhan                    
3         Travel history to Italy and Austria                    
4  Travel history to Dubai, Singapore contact                    

[5 rows x 26 columns]

# Data cleaning and transformations

In [7]:
# shape of dataframe
# ==================

p_df.shape

(10100, 26)

In [8]:
# columns
# =======

p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes', '', '', '', '', '', '', ''],
      dtype='object')

In [9]:
# selecting important columns only
# ================================

p_df = p_df.loc[:, :'Backup Notes']

In [10]:
# looking for missing values 
# ==========================

p_df.isna().sum()

Patient Number                               0
State Patient Number                         0
Date Announced                               0
Age Bracket                                  0
Gender                                       0
Detected City                                0
Detected District                            0
Detected State                               0
State code                                   0
Current Status                               0
Notes                                        0
Contracted from which Patient (Suspected)    0
Nationality                                  0
Type of transmission                         0
Status Change Date                           0
Source_1                                     0
Source_2                                     0
Source_3                                     0
Backup Notes                                 0
dtype: int64

In [11]:
# no. of empty strings in each column
# ===================================

print(p_df.shape)

for i in p_df.columns:
    print(i, '\t', p_df[p_df[i]==''].shape[0])

(10100, 19)
Patient Number 	 1491
State Patient Number 	 8573
Date Announced 	 1646
Age Bracket 	 8920
Gender 	 8143
Detected City 	 8980
Detected District 	 3350
Detected State 	 1647
State code 	 1647
Current Status 	 1647
Notes 	 2838
Contracted from which Patient (Suspected) 	 8776
Nationality 	 8904
Type of transmission 	 7265
Status Change Date 	 1756
Source_1 	 1852
Source_2 	 7900
Source_3 	 9795
Backup Notes 	 9740


In [12]:
# replacing empty strings with np.nan
# ==================================-

print(p_df.shape)

p_df = p_df.replace(r'', np.nan, regex=True)
p_df.isna().sum()

(10100, 19)


Patient Number                               1491
State Patient Number                         8573
Date Announced                               1646
Age Bracket                                  8920
Gender                                       8143
Detected City                                8980
Detected District                            3350
Detected State                               1647
State code                                   1647
Current Status                               1647
Notes                                        2838
Contracted from which Patient (Suspected)    8776
Nationality                                  8904
Type of transmission                         7265
Status Change Date                           1756
Source_1                                     1852
Source_2                                     7900
Source_3                                     9795
Backup Notes                                 9740
dtype: int64

In [13]:
# droping empty rows (row with just row number but without patient entry
# ======================================================================

p_df.dropna(subset=['Detected State'], inplace=True)
print(p_df.shape)
p_df.isna().sum()

(8453, 19)


Patient Number                                  0
State Patient Number                         6926
Date Announced                                  0
Age Bracket                                  7274
Gender                                       6497
Detected City                                7333
Detected District                            1704
Detected State                                  0
State code                                      0
Current Status                                  1
Notes                                        1192
Contracted from which Patient (Suspected)    7129
Nationality                                  7258
Type of transmission                         5619
Status Change Date                            110
Source_1                                      206
Source_2                                     6254
Source_3                                     8149
Backup Notes                                 8093
dtype: int64

In [14]:
p_df.columns

Index(['Patient Number', 'State Patient Number', 'Date Announced',
       'Age Bracket', 'Gender', 'Detected City', 'Detected District',
       'Detected State', 'State code', 'Current Status', 'Notes',
       'Contracted from which Patient (Suspected)', 'Nationality',
       'Type of transmission', 'Status Change Date', 'Source_1', 'Source_2',
       'Source_3', 'Backup Notes'],
      dtype='object')

In [15]:
# rename dateframe columns 
# ========================

p_df.columns = ['_'.join(col.lower().split()) for col in p_df.columns]
p_df.rename(columns = {'contracted_from_which_patient_(suspected)':'suspected_contacted_patient'}, inplace=True)
p_df.sample(5)

patient_number state_patient_number date_announced age_bracket gender  \
3025           3026                  NaN     03/04/2020         NaN    NaN   
3881           3882              TN-P531     05/04/2020         NaN      M   
2390           2391                  NaN     02/04/2020         NaN    NaN   
3297           3298                  NaN     04/04/2020         NaN    NaN   
7888           7889                  NaN     11/04/2020         NaN    NaN   

     detected_city detected_district     detected_state state_code  \
3025           NaN              Pune        Maharashtra         MH   
3881           NaN         Cuddalore         Tamil Nadu         TN   
2390           NaN           Krishna     Andhra Pradesh         AP   
3297           NaN               NaN              Delhi         DL   
7888           NaN           Kashmir  Jammu and Kashmir         JK   

     current_status               notes suspected_contacted_patient  \
3025   Hospitalized                 NaN                         NaN   
3881   Hospitalized  Travelled to Delhi                          E0   
2390   Hospitalized                 NaN                         NaN   
3297   Hospitalized                 NaN                         NaN   
7888   Hospitalized     Details awaited                         NaN   

     nationality type_of_transmission status_change_date  \
3025         NaN                  NaN         03/04/2020   
3881         NaN                Local         05/04/2020   
2390         NaN                  NaN         02/04/2020   
3297         NaN                  NaN         04/04/2020   
7888         NaN                  NaN         11/04/2020   

                                               source_1  \
3025  https://twitter.com/PTI_News/status/1246097072...   
3881                     State Bulletin: 05 Apr Evening   
2390  https://twitter.com/ArogyaAndhra/status/124570...   
3297                                                NaN   
7888  https://twitter.com/kansalrohit69/status/12489...   

                                               source_2  \
3025  https://experience.arcgis.com/experience/8167a...   
3881                                                NaN   
2390                                                NaN   
3297  https://twitter.com/PTI_News/status/1246409749...   
7888                                                NaN   

                                               source_3 backup_notes  
3025                                                NaN          NaN  
3881                                                NaN          NaN  
2390                                                NaN          NaN  
3297  https://twitter.com/AamAadmiParty/status/12464...          NaN  
7888                                                NaN          NaN

In [16]:
# creating patient id column from patient number
# ===============================================

p_df['p_id'] = p_df['patient_number'].apply(lambda x : 'P'+str(x))
p_df.columns

Index(['patient_number', 'state_patient_number', 'date_announced',
       'age_bracket', 'gender', 'detected_city', 'detected_district',
       'detected_state', 'state_code', 'current_status', 'notes',
       'suspected_contacted_patient', 'nationality', 'type_of_transmission',
       'status_change_date', 'source_1', 'source_2', 'source_3',
       'backup_notes', 'p_id'],
      dtype='object')

In [17]:
p_df = p_df.loc[:, :'backup_notes']

# Saving Data

In [18]:
p_df.sample(5)

patient_number state_patient_number date_announced age_bracket gender  \
3255           3256                  NaN     04/04/2020         NaN    NaN   
575             576                  NaN     25/03/2020         NaN    NaN   
4753           4754                  NaN     06/04/2020         NaN    NaN   
4598           4599                  NaN     06/04/2020         NaN    NaN   
7977           7978              TN-P952     11/04/2020         NaN    NaN   

     detected_city detected_district  detected_state state_code  \
3255           NaN               NaN           Delhi         DL   
575         Indore            Indore  Madhya Pradesh         MP   
4753           NaN             Betul  Madhya Pradesh         MP   
4598           NaN            Mumbai     Maharashtra         MH   
7977           NaN       Thiruvallur      Tamil Nadu         TN   

     current_status                     notes suspected_contacted_patient  \
3255   Hospitalized                       NaN                         NaN   
575    Hospitalized  Travelled from Rishikesh                         NaN   
4753   Hospitalized           Details awaited                         NaN   
4598   Hospitalized           Details awaited                         NaN   
7977   Hospitalized           Details awaited                         NaN   

     nationality type_of_transmission status_change_date  \
3255         NaN                  NaN         04/04/2020   
575        India                  TBD         25/03/2020   
4753         NaN                  NaN         06/04/2020   
4598         NaN                  NaN         06/04/2020   
7977         NaN                  NaN         11/04/2020   

                                               source_1  \
3255                                                NaN   
575   https://twitter.com/PTI_News/status/1242660625...   
4753  https://twitter.com/JansamparkMP/status/124718...   
4598  https://twitter.com/PTI_News/status/1247164710...   
7977  https://twitter.com/PTI_News/status/1248962069...   

                                               source_2  \
3255  https://twitter.com/PTI_News/status/1246409749...   
575   https://dbpost.com/mp-coronavirus-cases-update...   
4753                                                NaN   
4598  https://arogya.maharashtra.gov.in/pdf/covidmpr...   
7977                                                NaN   

                                               source_3 backup_notes  
3255  https://twitter.com/AamAadmiParty/status/12464...          NaN  
575                                                 NaN          NaN  
4753                                                NaN          NaN  
4598                                                NaN          NaN  
7977                                                NaN          NaN

In [19]:
p_df.to_csv('patients_data.csv', index=False)